In [1]:
import json
import matplotlib as mpl
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import re
import sys
import itertools
from collections import namedtuple, defaultdict
from pathlib import Path
from enum import Enum
from os.path import splitext

%matplotlib inline
%config InlineBackend.figure_format = 'retina'

%load_ext autoreload
%autoreload 1
%aimport analyze

with open('plots/style.json') as f:
    mpl.rcParams.update(json.load(f))

# Results

In [78]:
system = "frontier"
version = "v0.5.1"
oldversion = "v0.5.0"

## Current

In [79]:
new = analyze.Analysis(f"results/{system}/")
print(new)

Analysis for Celeritas v0.5.1-rc.1 on frontier


In [80]:
analyze.calc_geo_frac(new)

count      mean       std
problem                     geo    arch                           
atlas-tilecal               orange cpu     8.0  0.782562  0.001001
cms-hgcal                   orange cpu     8.0  0.731576  0.001200
testem15                    orange cpu     8.0  0.458680  0.009782
testem15+field              orange cpu     8.0  0.529762  0.003739
                                   gpu     8.0  0.521670  0.007683
testem15+field+msc          orange cpu     8.0  0.626237  0.005887
testem3-composite+field+msc orange cpu     8.0  0.767298  0.000208
                                   gpu     8.0  0.882681  0.003838
testem3-composite+msc       orange cpu     8.0  0.740070  0.001582
testem3-expanded+field+msc  orange cpu     8.0  0.768432  0.001475
testem3-flat                orange cpu     8.0  0.666687  0.001833
testem3-flat+field          orange cpu     8.0  0.701505  0.001258
testem3-flat+field+msc      orange cpu     8.0  0.783417  0.001049
                                   gpu     8.0  0.882589  0.003777
testem3-flat+msc            orange cpu     8.0  0.754198  0.002675

## Previous

In [81]:
old = analyze.Analysis(f"results-old/{system}/")
print(old, f"({oldversion})")

Analysis for Celeritas v0.5.0-dev.202+141cd4928 on frontier (v0.5.0)


In [82]:
ftab = analyze.make_failure_table(old.failures())
ftab.to_frame()

,Failure


In [83]:
analyze.calc_geo_frac(old)

count      mean       std
problem                     geo    arch                           
atlas-tilecal               orange cpu     8.0  0.688639  0.006343
cms-hgcal                   orange cpu     8.0  0.691758  0.003234
testem15                    orange cpu     8.0  0.420949  0.001738
testem15+field              orange cpu     8.0  0.532925  0.001342
                                   gpu     8.0  0.517098  0.000689
testem15+field+msc          orange cpu     8.0  0.636258  0.000632
testem3-composite+field+msc orange cpu     8.0  0.810377  0.002835
                                   gpu     8.0  0.902692  0.001695
testem3-composite+msc       orange cpu     8.0  0.784959  0.005576
testem3-expanded+field+msc  orange cpu     8.0  0.812374  0.003818
testem3-flat                orange cpu     8.0  0.535309  0.000774
testem3-flat+field          orange cpu     8.0  0.618430  0.000941
testem3-flat+field+msc      orange cpu     8.0  0.762960  0.000426
                                   gpu     8.0  0.902433  0.003229
testem3-flat+msc            orange cpu     8.0  0.723605  0.009448

# Analysis

## Compare manually

In [84]:
new_throughput = new.result['avg_event_per_time'].xs('vecgeom', level='geo').unstack('arch')

KeyError: 'vecgeom'

In [85]:
old_throughput = old.result['avg_event_per_time'].xs('vecgeom', level='geo').unstack('arch')

KeyError: 'vecgeom'

In [86]:
new_throughput / old_throughput - 1

arch                                       cpu    cpu+g4       gpu    gpu+g4  \
problem                     instance                                           
atlas-tilecal               0        -0.394953  0.003041 -0.044679  0.282126   
                            1        -0.400524  0.000088 -0.046422  0.279727   
                            2        -0.396896 -0.013201 -0.048575  0.271690   
                            3        -0.418273  0.003622 -0.049623  0.263174   
cms-hgcal                   0        -0.332316  0.034221 -0.025306  0.301586   
                            1        -0.344703  0.031886 -0.026230  0.315164   
                            2        -0.333422  0.000537 -0.028709  0.320760   
                            3        -0.333555  0.006087 -0.027231  0.306930   
cms2018+field+msc           0        -0.235427 -0.066730 -0.019336  1.998694   
                            1        -0.193855 -0.052122 -0.037959  2.079291   
                            2        -0.203617 -0.049475 -0.038628  2.023586   
                            3        -0.206178 -0.017616 -0.038818  2.048397   
cms2018                     0        -0.423404 -0.065421 -0.107250  0.382504   
                            1        -0.420653 -0.038170 -0.104166  0.382095   
                            2        -0.404716 -0.100255 -0.108218  0.387237   
                            3        -0.418164 -0.056405 -0.105804  0.389884   
testem15+field+msc          0        -0.284986 -0.015402  0.022763  0.164745   
                            1        -0.296401 -0.034992  0.022852  0.168074   
                            2        -0.317892  0.032344  0.006581  0.205858   
                            3        -0.295933 -0.009646  0.018564  0.221908   
testem15+field              0              NaN       NaN       NaN       NaN   
                            1              NaN       NaN       NaN       NaN   
                            2              NaN       NaN       NaN       NaN   
                            3              NaN       NaN       NaN       NaN   
testem3-composite+field+msc 0        -0.137864  0.012740  0.001638  0.966524   
                            1        -0.098270 -0.013259 -0.008220  0.952655   
                            2        -0.173837 -0.049993 -0.002114  0.972260   
                            3        -0.106605 -0.020956  0.001192  0.953743   
testem3-composite+msc       0        -0.215660 -0.043107 -0.006139  0.340269   
                            1        -0.178468 -0.038798 -0.004822  0.386608   
                            2        -0.170865 -0.046098 -0.002867  0.341149   
                            3        -0.214332 -0.042409 -0.004640  0.343474   
testem3-expanded+field+msc  0        -0.118571 -0.003213  0.001084  0.948978   
                            1        -0.131644 -0.017729 -0.009769  0.938065   
                            2        -0.133145 -0.012107 -0.006338  0.944984   
                            3        -0.141017 -0.006871  0.000473  0.955821   
testem3-flat+field+msc      0        -0.151696 -0.019336  0.002829  1.053366   
                            1        -0.165919 -0.015619  0.001331  1.044966   
                            2        -0.145030  0.002821 -0.003402  1.037445   
                            3        -0.185116 -0.042127 -0.000327  1.053446   
testem3-flat                0        -0.448280  0.037532 -0.004140  0.441387   
                            1        -0.396700 -0.009817  0.003495  0.403694   
                            2        -0.447638  0.019040 -0.000798  0.402904   
                            3        -0.466701  0.014791 -0.002949  0.402663   

arch                                  gpu+sync  
problem                     instance            
atlas-tilecal               0              NaN  
                            1              NaN  
                            2              NaN  
                            3              NaN  
cms-hgcal                

## Performance drop between versions

In [87]:
#analyses = {a.version: a for a in [frontier, frontier_v050]}
analyses = {
    oldversion: old,
    version: new,
}
throughput = {k: a.result['avg_event_per_time'] for k, a in analyses.items()}
plot_rel_to = old

In [88]:
def plot_ratio(result_dict):
    rel = analyze.summarize_instances(result_dict[version] / result_dict[oldversion])
    rel.dropna(inplace=True)
    fig, ax = plt.subplots(layout="constrained")
    plot_rel_to.plot_results(ax, rel)
    ax.grid(which='both')
    ax.legend()
    ax.text(0.98, 0.02,
        f"{version} / {oldversion} - 1 on {system}",
        va='bottom', ha='right',
        fontstyle='italic', color=(0.5,)*3, size='xx-small',
        transform=ax.transAxes,
        zorder=-100
    )

    return ax

In [89]:
steps = {k: a.result['avg_steps_per_primary'] for k, a in analyses.items()}

In [90]:
ax = plot_ratio(throughput)
ax.set_ylabel(f"Throughput [events/sec]")
ax.get_figure().savefig(f"results-old/{system}/rel-throughput.png", dpi=300)
ax.get_figure().savefig(f"results-old/{system}/rel-throughput.pdf", transparent=True)
plt.close()

In [91]:
ax = plot_ratio(steps)
ax.set_ylabel(f"Work [steps/primary]")
ax.get_figure().savefig(f"results-old/{system}/rel-work.png", dpi=300)
ax.get_figure().savefig(f"results-old/{system}/rel-work.pdf", transparent=True)
plt.close()